In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
0,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,Female,29,Bangalore,,I am a working professional in IT sector (IT+F...,My parents are quite supportive of my career a...,Never married,No,Between 12-15 Lakhs,IT Professional ( Business Analysis),...,"I am in Bangalore currently because of my job,...",I give consent,Hindu,I am not,It will not work for me,No,Linkedin,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
491,Male,39,Banglore,,"Friends, A job I love",Good,"Separated, filed for a divorce",Yes,Under 12 Lakhs,"Music Educator, Violinist, Music Composer",...,Nothing Special,I give consent,Christian,I am not sure,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
492,Male,30,Mumbai,,Being able to save lives and see happiness on ...,I had an awesome childhood where i grew up in ...,Never married,No,Under 12 Lakhs,"I am a doctor,currently pursuing my post gradu...",...,I would like to meet someone who is passionate...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
493,Female,33,Mumbai,,We live in times where we should be aware a

In [3]:
dataset.to_csv("dataset.csv")

In [4]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
2,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Female,29,Bangalore,,I am a working professional in IT sector (IT+F...,My parents are quite supportive of my career a...,Never married,No,Between 12-15 Lakhs,IT Professional ( Business Analysis),...,"I am in Bangalore currently because of my job,...",I give consent,Hindu,I am not,It will not work for me,No,Linkedin,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
493,Male,39,Banglore,,"Friends, A job I love",Good,"Separated, filed for a divorce",Yes,Under 12 Lakhs,"Music Educator, Violinist, Music Composer",...,Nothing Special,I give consent,Christian,I am not sure,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
494,Male,30,Mumbai,,Being able to save lives and see happiness on ...,I had an awesome childhood where i grew up in ...,Never married,No,Under 12 Lakhs,"I am a doctor,currently pursuing my post gradu...",...,I would like to meet someone who is passionate...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
495,Female,33,Mumbai,,We live in times where we should be aware a

In [5]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [6]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
492,Female,29,Bangalore,Never married
493,Male,39,Banglore,"Separated, filed for a divorce"
494,Male,30,Mumbai,Never married
495,Female,33,Mumbai,Never married


In [7]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [9]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [10]:
encoded_categorical_features

,Legally Separated,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
492,0,1,0,0,0,0,0
493,0,0,1,0,0,0,0
494,0,1,0,0,0,0,0
495,0,1,0,0,0,0,0


In [11]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [12]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
492,Female,29,Bangalore,Never married
493,Male,39,Banglore,"Separated, filed for a divorce"
494,Male,30,Mumbai,Never married
495,Female,33,Mumbai,Never married


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [14]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
492,Female,29,Bangalore,Never married
493,Male,39,Banglore,"Separated, filed for a divorce"
494,Male,30,Mumbai,Never married
495,Female,33,Mumbai,Never married


In [15]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [17]:
print(features.shape)

(495, 170)


In [18]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [19]:
row =210
selectedRow = refDataset.iloc[row]

In [20]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 401 252 409 424 396 463 316  11 108 460 372 272 342 165
  266  45 440 313  25 331 320  41 330 385 278 326 332 383 247 245 343  48
  351 290 156 261  24 311  81 229 386  26 139 256 452 449 181 358 425 309
  288 363 123 354 333  51 390  54  30 122  97 118 110 376 295 128 462 262
   86  84 412 136 381  19 260 231 349 454 414 325 259  55 446 274  75 336
  365 307 310 114 393 183  42 287 265 355 375 308 200 415  52 353 100 378
   32   1 404 441  73  50 469  18 456 105 447 457 327 185 174  27   6  29
  344 258 251  12 111  65 178 301 453 228 388 282 373 467 221 291 246 345
   77 296  66 472 443 420  96 406 435 121 166  56 348 244 201  83 428 196
  237 169  36 135 380 411 106 160 328 209 340  91  89 241 194 143 432 321
  352 357 171 255  71 219 163 335 208 464 230 379 421 107  76 184 264 438
  104 112 271  21 238 314 147 422 173 211 119 444 249 413 142  53 382 442
   35 263 205 214 403 408  23  62  70  79 391 191  94 137 242 141  59  37
  315 180 257 152 346 101 429 188 426 

In [21]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 47 'New Delhi ' 'Never married']


['Male' 42 'New Delhi ' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 50 'Delhi ' 'Separated, not legally divorced']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Bangalore, Karnataka, India' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Widow/ Widower']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Mumbai ' 'Never married']


['Male' 43 'Tiruvalla ' 'Never married']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 42 'Bangalore ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 36 'Delhi' 'Never married']


['Male' 55 'Gurgaon' 'Separated, not legally divorced']


['

In [22]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [23]:
print(preprocessData(refDataset))

[[ 1.         -0.71385894  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.20322285  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.88407098  0.         ...  0.          0.
   0.        ]
 ...
 [ 1.         -1.22449504  0.         ...  0.          0.
   0.        ]
 [ 0.         -0.71385894  0.         ...  0.          0.
   0.        ]
 [ 0.         -0.03301082  0.         ...  0.          0.
   0.        ]]


In [24]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']